In [ ]:
from google.colab import drive
import os
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag

In [ ]:
drive.mount('/content/drive')
datadir = "/content/drive/My Drive/CS546Data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_files_to_json(folder_paths):
    data_dict = {}


    if os.path.exists('/content/minecraft_data.json'):
        with open('/content/minecraft_data.json', 'r', encoding='utf-8') as json_file:
            data_dict = json.load(json_file)

    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):

                topic = filename.replace(".txt", "")


                with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                    content = file.read()


                data_dict[topic] = {
                    'description': content

                }


    data_json = json.dumps(data_dict, indent=4)


    with open('/content/minecraft_data.json', 'w', encoding='utf-8') as json_file:
        json_file.write(data_json)

    return data_dict

folder_paths = [
    datadir + 'downloaded_pages/Block/Normal_Blocks',
    datadir + 'downloaded_pages/Block/Technical_Blocks',
    # datadir + 'downloaded_pages/Biome/Woodland biomes',
    # datadir + 'downloaded_pages/Biome/Wetland biomes',
    # datadir + 'downloaded_pages/Biome/Void biomes',
    # datadir + 'downloaded_pages/Biome/The End biomes',
    # datadir + 'downloaded_pages/Biome/Snowy biomes',
    # datadir + 'downloaded_pages/Biome/Offland biomes',
    # datadir + 'downloaded_pages/Biome/Nether biomes',
    # datadir + 'downloaded_pages/Biome/Highland biomes',
    # datadir + 'downloaded_pages/Biome/Flatland biomes',
    # datadir + 'downloaded_pages/Biome/Arid-land biomes',
    # datadir + 'downloaded_pages/Items/Items_for_Use',
    # datadir + 'downloaded_pages/Items/Items_for_Indirect_Use',
    # datadir + 'downloaded_pages/Items/Items_for_Creation',
    # datadir + 'downloaded_pages/Gameplay/Trading',
    # datadir + 'downloaded_pages/Gameplay/Smithing',
    # datadir + 'downloaded_pages/Gameplay/Smelting',
    # datadir + 'downloaded_pages/Gameplay/Redstone_circuits',
    # datadir + 'downloaded_pages/Gameplay/Enchanting',
    # datadir + 'downloaded_pages/Gameplay/Crafting',
    # datadir + 'downloaded_pages/Gameplay/Brewing',
    # datadir + 'downloaded_pages/Gameplay/Archaeology',
    # datadir + 'downloaded_pages/Effect',
    # datadir + 'downloaded_pages/Mob/Passive_Mobs',
    # datadir + 'downloaded_pages/Mob/Neutral_Mobs',
    # datadir + 'downloaded_pages/Mob/Hostile_Mobs',
    # datadir + 'downloaded_pages/Mob/Defensive_Mobs',
    # datadir + 'downloaded_pages/Mob/Boss_Mobs',



]
minecraft_data = read_files_to_json(folder_paths)

KeyboardInterrupt: ignored

In [ ]:
!pip install cohere
!pip install tiktoken
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 43.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.4 MB/s eta 0:00:00
 

In [ ]:
import json
from openai import OpenAI

client = OpenAI(
)

def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def find_answer_start(context, answer):
    return context.find(answer)

def generate_qa_pairs(context, max_questions=3):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Generate {max_questions} question, answer, and answer start index pairs based on the following text:\n\n{context}\n\nFor each answer, provide the starting index of the answer in the context, and the answer should be derived from the text directly without any modification, and it should not be pure numbers.\n The questions should look like this: 'Question 1: (Question here)', answer should look like this: 'Answer 1: (Answer here)', and the answer start index should look like this: 'Answer start index 1: (answer start index here)'",
                    # "content": f"Generate {max_questions} question, answer, and answer start index pairs based on the following text:\n\n{context}\n\nFor each answer, provide the starting index of the answer in the context, and the answer should be strictly derived from the text directly without any modification, and should not be a numerical answer.\nBe advised, some words might be surrounded by 'entity', it might look like this: entity cauldron entity. When you encounter this in the text, treat them as is and put 'entity' in the answer, this is a tagging step I applied to the text that I needed later.\n The questions should look like this: 'Question 1: (Question here)', and answer should look like this: 'Answer 1: (Answer here)'",
                }
            ],
            model="gpt-4-1106-preview"
        )
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# preprocessed_data = load_data('./preprocessed_minecraft_data.json')
original_data = load_data('./minecraft_data.json')

final_data = {"data": []}

for title, content in original_data.items():

    preprocessed_context = content['description']
    qa_pairs = generate_qa_pairs(preprocessed_context, max_questions=3)

    qas = []
    for qa in qa_pairs.split('\n\n'):
        try:
            parts = qa.split('\n')
            if len(parts) >= 3:

                question = parts[0].split(': ')[1]
                answer = parts[1].split(': ')[1]
                print(question)
                # print(answer)
                answer_start_index_line = parts[2]
                print(answer_start_index_line)
                answer_start_index = int(answer_start_index_line.split(': ')[1])


                answer_start = answer_start_index

                qas.append({
                    "question": question,
                    "id": f"q{len(qas)+1}",
                    "answers": [
                        {
                            "text": answer,
                            "answer_start": answer_start
                        }
                    ]
                })
        except Exception as e:
            print(f"An error occurred while parsing Q&A for {title}: {e}")
            continue

    preprocessed_context = original_data[title]['description']
    final_data["data"].append({
        "title": title,
        "paragraphs": [
            {
                "context": preprocessed_context,
                "qas": qas
            }
        ]
    })
    # print(final_data)

with open('final_minecraft_dataset.json', 'w', encoding='utf-8') as outfile:
    json.dump(final_data, outfile, indent=4)
print("finished")

ModuleNotFoundError: ignored

In [ ]:
!pip install pandas

In [ ]:
import json
import pandas as pd

with open('./part_scraped_final_minecraft_dataset.json', 'r') as file:
    data = json.load(file)

qa_data = []
for entry in data['data']:
    title = entry['title']
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            id = qa['id']
            for answer in qa['answers']:
                answer_text = answer['text']
                answer_start = answer['answer_start']
                qa_data.append({'title': title, 'context': context, 'question': question, 'id': id, 'answer_text': answer_text, 'answer_start': answer_start})

df = pd.DataFrame(qa_data)

In [ ]:
df

,title,context,question,id,answer_text,answer_start
0,Sponge,Overview\n A sponge is a block that can be ...,How can a wet sponge be turned back into a nor...,q1,"A wet sponge can be dried in a furnace, making...",1067
1,Sponge,Overview\n A sponge is a block that can be ...,Where can sponges be found in Minecraft?,q2,Sponges can only be found in ocean monuments.,250
2,Sponge,Overview\n A sponge is a block that can be ...,What is the maximum distance a sponge can abso...,q3,A sponge absorbs both flowing and source block...,1935
3,Leaves,Overview\n Leaves are natural blocks that g...,What are the default tools for breaking leaves...,q1,Hoes are the default tools for breaking leaves.,114
4,Leaves,Overview\n Leaves are natural blocks that g...,Which blocks are affected by the biome dyeing ...,q2,Leaves are affected by the biome dyeing algori...,5351
...,...,...,...,...,...,...
1082,End_Portal_(block),Overview\n The end portal block is a block ...,How can end portal blocks be placed in Java Ed...,q2,It can be placed only by using block placement...,270
1083,End_Portal_(block),Overview\n The end portal block is a block ...,What happens when entities travel to the Overw...,q3,Players traveling to the Overworld appear at t...,829
1084,Bubble_Column,Overview\n A bubble column is a non-solid b...,What are the two ways in which a bubble column...,q1,A bubble column is a non-solid block generated...,31
1085,Bubble_Column,Overview\n A bubble column is a non-solid b...,How can players and mobs make use of bubble co...,q2,Players and air-breathing mobs can enter a bub...,1180


In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_qa_pair(context, question):
    input_ids = tokenizer.encode(question, context)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    return input_ids, tokens

sample_context = df.iloc[0]['context']
sample_question = df.iloc[0]['question']
input_ids, tokens = tokenize_qa_pair(sample_context, sample_question)

Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
def sliding_window(context, question, max_len=512, stride=128):
    question_tokens = tokenizer.encode(question, add_special_tokens=False)
    context_tokens = tokenizer.encode(context, add_special_tokens=False)

    max_context_length = max_len - len(question_tokens) - 3

    chunks = []
    start = 0
    while start < len(context_tokens):
        end = start + max_context_length
        end = min(end, len(context_tokens))

        chunk = context_tokens[start:end]

        input_ids = [tokenizer.cls_token_id] + question_tokens + [tokenizer.sep_token_id] + chunk + [tokenizer.sep_token_id]

        chunks.append(input_ids)

        start += stride

    return chunks

sample_context = df.iloc[0]['context']
sample_question = df.iloc[0]['question']
chunks = sliding_window(sample_context, sample_question)

In [ ]:
def adjust_answer_positions(context_tokens, chunks, answer_start, answer_end):
    adjusted_positions = []
    context_start_pos = 0

    for chunk in chunks:
        chunk_end_pos = context_start_pos + len(chunk)

        if context_start_pos <= answer_start < chunk_end_pos or context_start_pos < answer_end <= chunk_end_pos:
            new_start = max(0, answer_start - context_start_pos)
            new_end = min(len(chunk), answer_end - context_start_pos)
            adjusted_positions.append((new_start, new_end))
        else:
            adjusted_positions.append((None, None))

        context_start_pos += len(chunk)

    return adjusted_positions

In [ ]:
def prepare_data_with_sliding_window(df, max_len=512, stride=128):
    prepared_data = []

    for index, row in df.iterrows():
        context = row['context']
        question = row['question']
        answer_text = row['answer_text']
        answer_start = row['answer_start']
        answer_end = answer_start + len(answer_text)

        # Tokenize the context
        context_tokens = tokenizer.encode(context, add_special_tokens=False)

        # Apply sliding window
        chunks = sliding_window(context, question, max_len, stride)

        # Adjust answer positions for each chunk
        adjusted_positions = adjust_answer_positions(context_tokens, chunks, answer_start, answer_end)

        for chunk, (new_start, new_end) in zip(chunks, adjusted_positions):
            if new_start is not None and new_end is not None:
                prepared_data.append({
                    'input_ids': chunk,
                    'answer_start': new_start,
                    'answer_end': new_end,
                })

    return prepared_data

In [ ]:
prepared_data = prepare_data_with_sliding_window(df)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids = torch.tensor(item['input_ids'], dtype=torch.long)
        length = len(input_ids)  # Length of the sequence
        start_positions = torch.tensor(item['answer_start'], dtype=torch.long)
        end_positions = torch.tensor(item['answer_end'], dtype=torch.long)


        return input_ids, length, start_positions, end_positions

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# def collate_fn(batch):
#     # Debugging: print the batch structure
#     # print(f"Batch received in collate_fn: {batch}")

#     input_ids, lengths, start_positions, end_positions = zip(*batch)

#     # Pad the input sequences
#     input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

#     # Convert other elements to tensors
#     lengths = torch.tensor(lengths)
#     start_positions = torch.tensor(start_positions)
#     end_positions = torch.tensor(end_positions)

#     return input_ids_padded, lengths, start_positions, end_positions
def collate_fn(batch):
    input_ids, lengths, start_positions, end_positions = zip(*batch)

    # Pad the input sequences
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

    # Create attention masks
    attention_masks = (input_ids_padded != tokenizer.pad_token_id).long()

    # Convert start_positions and end_positions to tensors
    start_positions = torch.tensor(start_positions)
    end_positions = torch.tensor(end_positions)

    return input_ids_padded, attention_masks, start_positions, end_positions

In [ ]:
qa_dataset = QADataset(prepared_data)

In [ ]:
qa_dataset[0]

(tensor([  101,  2129,  2064,  1037,  4954, 25742,  2022,  2357,  2067,  2046,
          1037,  3671, 25742,  1029,   102,  1996,  2051,  1996, 25742, 28123,
          2049, 17462,  1010,  1996,  2300, 18916,  2046,  1996, 13610,  1010,
          2975,  1037, 13610,  1997,  2300,  1999,  1996,  4762, 10453,  1012,
          4728,  1010,  1996,  2300,  2074,  9345, 17822,  8520,  1012,  1000,
         17462,  1037,  4954, 25742,  2872,  1999,  1996,  5658,  5886,  2852,
          3111,  2041,  6880,  2007,  1037, 23893,  1997,  5492,  1998,  4332,
          2046,  1037,  3671, 25742,  1012,  1037,  4954, 25742,  2872,  1999,
          2151,  4318, 16012,  4168,  1999,  1996,  2058, 11108,  2852,  3111,
          2041,  2044,  1037,  2261,  2781,  1998,  4332,  2046,  1037,  3671,
         25742,  1012,  1031, 28272,  3179,  2069,  1033,  2065,  1037,  3671,
         25742,  3310,  2046,  3967,  2007,  2300,  1999,  1037,  4318, 16012,
          4168,  1010,  2009, 16888,  2015,  1996,  

In [ ]:
dataloader = DataLoader(qa_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [ ]:
import torch
from torch.optim import AdamW
from transformers import BertForQuestionAnswering

accumulation_steps = 2  # For example, to mimic a batch size twice as large as currently fits in memory
# Load your model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Number of training epochs
num_epochs = 15

# Training loop
model.train()
optimizer.zero_grad()  # Zero out gradients initially

# for epoch in range(num_epochs):
#     total_loss = 0
#     for step, batch in enumerate(dataloader):
#         input_ids, _, start_positions, end_positions = [b.to(device) for b in batch]

#         # Forward pass
#         outputs = model(input_ids=input_ids, start_positions=start_positions, end_positions=end_positions)
#         loss = outputs.loss / accumulation_steps  # Scale loss
#         total_loss += loss.item()

#         # Backward pass
#         loss.backward()

#         # Perform an optimization step every 'accumulation_steps' mini-batches
#         if (step + 1) % accumulation_steps == 0:
#             optimizer.step()
#             optimizer.zero_grad()  # Zero gradients after optimization

#     print(f"Epoch {epoch + 1}: Loss = {total_loss / len(dataloader)}")
for epoch in range(num_epochs):
    total_loss = 0
    for step, batch in enumerate(dataloader):
        # Unpack the batch data and move to the correct device
        input_ids, attention_masks, start_positions, end_positions = [b.to(device) for b in batch]

        # Forward pass: pass attention_mask to the model as well
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, start_positions=start_positions, end_positions=end_positions)

        # Scale loss to account for gradient accumulation
        loss = outputs.loss / accumulation_steps
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Perform an optimization step every 'accumulation_steps' mini-batches
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()  # Zero gradients after optimization

    print(f"Epoch {epoch + 1}: Loss = {total_loss / len(dataloader)}")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Loss = 2.9099140380456197
Epoch 2: Loss = 2.7761252081491112
Epoch 3: Loss = 2.6050129363207315
Epoch 4: Loss = 2.2538014669728472
Epoch 5: Loss = 1.6527079382563026
Epoch 6: Loss = 1.0465578470772845
Epoch 7: Loss = 0.7175250591301336
Epoch 8: Loss = 0.5288648411510436
Epoch 9: Loss = 0.36015772780146055
Epoch 10: Loss = 0.27147316690382917
Epoch 11: Loss = 0.2095160826101778
Epoch 12: Loss = 0.16970092099826267
Epoch 13: Loss = 0.16083209533471524
Epoch 14: Loss = 0.13649040070642543
Epoch 15: Loss = 0.11887616505543512


In [ ]:
model_save_path = './saved_model_directory'
model.save_pretrained(model_save_path)

In [ ]:
tokenizer_save_path = './saved_tokenizer_directory'
tokenizer.save_pretrained(tokenizer_save_path)

('./saved_tokenizer_directory/tokenizer_config.json',
 './saved_tokenizer_directory/special_tokens_map.json',
 './saved_tokenizer_directory/vocab.txt',
 './saved_tokenizer_directory/added_tokens.json')

In [ ]:
from transformers import BertTokenizer

# Load the trained model and tokenizer
model = BertForQuestionAnswering.from_pretrained('./saved_model_directory')
tokenizer = BertTokenizer.from_pretrained('./saved_tokenizer_directory')
model.to(device)
model.eval()

# Function to ask a question to the model
def ask_question(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length=512, truncation=True)
    inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits

    # Get the most likely beginning and end of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1  # Add 1 to get the inclusive end index

    # Convert the token indices to actual tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    answer = tokenizer.convert_tokens_to_string(tokens[answer_start:answer_end])

    return answer

In [ ]:
context = "Overview: A sponge is a block that can be placed to remove water in an area around itself. This turns it into a wet sponge, which can be smelted or placed in the Nether to dry it to be reused. Sponges can only be found in ocean monuments.\n\nObtaining\n    Dry sponge does not generate in the world.\n\n  Breaking\n    Either type of sponge can be mined by hand, or with any tool, dropping itself as an item; however, hoes break sponges the fastest compared to other tools.\n\n    Block,Sponge Wet Sponge\n\n    Hardness,0.6\n\n    Tool,Hoe\n\n    Breaking time (sec) [A]\n\n    Default,0.9\n\n    Wooden,0.45\n\n    Stone,0.25\n\n    Iron,0.15\n\n    Diamond,0.15\n\n    Netherite,0.1\n\n    Golden,0.1\n\n\n  Natural generation\n    Ocean monuments can generate \"sponge rooms.\" Each room contains an average of 30 wet sponges. See Ocean Monument/Structure for details.\n\n  Mob loot\n    An elder guardian always drops one wet sponge when killed by the player.\n\n  Smelting\n    A wet sponge can be dried in a furnace, making the sponge reusable.\n\n    Ingredients,Smelting recipe,Description\n\n    Wet sponge + any fuel,0.15,\"If a bucket is in the fuel slot at the time the sponge completes its drying, the water drains into the bucket, leaving a bucket of water in the fuel slot. Otherwise, the water just evaporates.\"\n\n\n  Drying\n    A wet sponge placed in the Nether dries out instantly with a puff of steam and turns into a normal sponge. A wet sponge placed in any dry biome in the Overworld dries out after a few minutes and turns into a normal sponge.\u200c[Bedrock Edition  only] If a normal sponge comes into contact with water in a dry biome, it absorbs the water and immediately dries out with a puff of steam.\u200c[Bedrock Edition  only]\n\nUsage\n    A sponge can be used to turn water into air by \"absorbing\" the water. A sponge instantly absorbs nearby water when it is placed next to water or when water comes into contact with it. Sponges immediately turn wet upon absorption and do not absorb any more water afterwards. Wet sponges drip small water particles. Sponges in item form do not absorb water or become wet. A sponge absorbs both flowing and source blocks of water up to 7 blocks away (taken as a taxicab distance) in all six directions around itself. Instead of a cube or a sphere, this effectively encloses the volume of an octahedron with vertices placed 7 units away from the center in each direction due to the game's extensive use of taxicab distance. A sponge does not absorb more than 65 blocks of water however, and water closest to the sponge is absorbed first. The absorption propagates only between adjacent water blocks and does not \"jump over\" non-water blocks, including air."
question = "Where can sponges be found in Minecraft?"
predicted_answer = ask_question(context, question)
print("Q:", question)
print("A:", predicted_answer)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: Where can sponges be found in Minecraft?
A: + any fuel , 0 . 15 , " if a bucket is in the fuel slot at the time the sponge completes its drying , the water drains into the bucket , leaving a bucket of water in the fuel slot . otherwise , the


In [ ]:
context = "Overview\n    Wheat seeds (in Java Edition) or seeds (in Bedrock Edition) are items obtained by breaking grass, or more abundantly harvested from wheat crops, and are used to plant them. Wheat crops are planted in farmland and used to grow wheat and wheat seeds.\n\nObtaining\n  Breaking\n    Harvesting fully-grown wheat crops yields from 1 to 4 seeds per crop harvested (about 2\u00a05\u20447 seeds/crop harvested on average). Wheat seeds can be obtained from breaking all variants of grass, which yields 0 to 1 seed. If harvested with a Fortune enchanted tool the drop rate from grass is increased. The looting is calculated by a binomial distribution: a drop is attempted three times with a success rate of around 57% to yield the 0\u20133 drops. Each level of Fortune enchantment increases the number of attempts by one.\n\n    Resource location,Source,Drops\n\n    Wheat,Wheat Seeds\n\n    Default tool,With Fortune I,With Fortune II,With Fortune III\n\n    blocks/wheat,Wheat (age 0\u20136),0,1,1,1,1\n\n    Wheat (age 7),1,1 (~7.87%) or 2 (~31.49%) or 3 (~41.98%) or 4 (~18.66%),1 (~3.37%) or 2 (~17.99%) or 3 (~35.98%) or 4 (~31.99%) or 5 (~10.66%),1 (~1.44%) or 2 (~9.64%) or 3 (~25.70%) or 4 (~34.27%) or 5 (~22.85%) or 6 (~6.09%),1 (~0.62%) or 2 (~4.96%) or 3 (~16.52%) or 4 (~29.38%) or 5 (~29.38%) or 6 (~15.67%) or 7 (~3.48%)\n\n\n  Natural generation\n    Village farm plots have a chance of being wheat crops. The exact chance depends on the style of the village:\n\n    Village style,Chance\n\n    Savanna,80%\n\n    Desert,70%\n\n    Taiga,60%\n\n    Plains,50%\n\n    Snowy,20%\n\n\n  Chest loot\n    Item,Structure,Container,Quantity,Chance\n\n    Java Edition\n\n    Wheat Seeds,Trail ruins,Suspicious\u00a0gravel,1,\"2.2% { \"\"item\"\": \"\"Wheat Seeds\"\", \"\"stacksize\"\": 1, \"\"chance\"\": 0.022222222222222254, \"\"structure\"\": \"\"Trail ruins\"\", \"\"container\"\": \"\"Suspicious gravel\"\" }\"\n\n    Village,Fisherman's\u00a0chest,1\u20133,\"57.5% { \"\"item\"\": \"\"Wheat Seeds\"\", \"\"stacksize\"\": \"\"1\u20133\"\", \"\"chance\"\": 0.5751805328746795, \"\"structure\"\": \"\"Village\"\", \"\"container\"\": \"\"Fisherman's chest\"\" }\"\n\n    Savanna\u00a0house\u00a0chest,1\u20135,\"71.7% { \"\"item\"\": \"\"Wheat Seeds\"\", \"\"stacksize\"\": \"\"1\u20135\"\", \"\"chance\"\": 0.7169464744127282, \"\"structure\"\": \"\"Village\"\", \"\"container\"\": \"\"Savanna house chest\"\" }\"\n\n    Bedrock Edition\n\n    Wheat Seeds,Trail ruins,Suspicious\u00a0gravel,1,\"2.2% { \"\"item\"\": \"\"Wheat Seeds\"\", \"\"stacksize\"\": 1, \"\"chance\"\": 0.021739130434782594, \"\"structure\"\": \"\"Trail ruins\"\", \"\"container\"\": \"\"Suspicious gravel\"\" }\"\n\n    Village,Savanna\u00a0house\u00a0chest,1\u20135,\"71.7% { \"\"item\"\": \"\"Wheat Seeds\"\", \"\"stacksize\"\": \"\"1\u20135\"\", \"\"chance\"\": 0.7169464744127282, \"\"structure\"\": \"\"Village\"\", \"\"container\"\": \"\"Savanna house chest\"\" }\"\n\n\n  Trading\n    Wandering traders sell wheat seeds for an emerald.\n\n  Villager gifts\n    Nitwit and unemployed villagers throw wheat seeds at players under the Hero of the Village effect.\n\nUsage\n  Crop\n    Wheat seeds can be placed on farmland by right-clicking, where they grow through eight stages. When left alone, wheat seeds planted on farmland grow to become wheat crops, which can be harvested by the player. Planted seeds require a light level of 9 or greater to continue growing. If the light level is 7 or below, the crops instantly un-plant themselves (\"pop off\"). It is not possible to plant seeds if the light level is too low. Crops grow faster if the farmland they are planted in is hydrated. Using bone meal on crops also increases the speed of growth by randomly increasing their growth stage by 2 to 5. Breaking the final stage produces 1 to 4 wheat seeds (or more with Fortune) and 1 wheat. If they are harvested early, they drop 1 seed without any wheat. Crops break if pushed by a piston or if their supporting farmland breaks or turns to dirt (i.e. by being trampled), dropping their usual drops.\n\n  Breeding\n    Like other seeds, wheat seeds can be used to breed chickens and reduce the remaining growth duration of baby chickens by 10%. Chickens also follow a player holding wheat seeds.\n\n  Taming\n    Like other seeds, wheat seeds can be used to tame parrots.\n\n  Composting\n    Placing wheat seeds into a composter has a 30% chance of raising the compost level by 1. A stack of wheat seeds yields an average of 2.74 bonemeal."
question = "What do wheat seeds turn into when left alone to grow on farmland?"
predicted_answer = ask_question(context, question)
print("Q:", question)
print("A:", predicted_answer)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: What do wheat seeds turn into when left alone to grow on farmland?
A: ~ 9 . 64 % ) or 3 ( ~ 25 . 70 % ) or 4 ( ~ 34 . 27 % ) or 5 ( ~ 22 . 85 % ) or 6 ( ~ 6 . 09 % ) , 1


In [ ]:
context = "Overview\n    Stone bricks are materials found in various structures. They come in three other variants, mossy stone bricks, cracked stone bricks, and chiseled stone bricks.\n\nObtaining\n  Breaking\n    Stone bricks can be mined using any pickaxe. If mined without a pickaxe, they drop nothing.\n\n    Block,Stone Bricks\n\n    Hardness,1.5\n\n    Tool,wooden pickaxe\n\n    Breaking time (sec) [A]\n\n    Default,7.5\n\n    Wooden,1.15\n\n    Stone,0.6\n\n    Iron,0.4\n\n    Diamond,0.3\n\n    Netherite,0.25\n\n    Golden,0.2\n\n\n  Infested blocks\n    Stone bricks can also be obtained by mining their respective infested blocks with Silk Touch.\n\n  Natural generation\n    Regular stone bricks generate as part of strongholds, making up most of the walls, ceilings, and floors. They can also generate as part of ocean ruins and ruined portals. Igloo basements are lined with stone bricks. Stone bricks can also spawn in trail ruins.\n\n  Chest loot\n    Item,Structure,Container,Quantity,Chance\n\n    Java Edition\n\n    Stone Bricks,Village,Mason's\u00a0chest,1,\"37.7% { \"\"item\"\": \"\"Stone Bricks\"\", \"\"stacksize\"\": 1, \"\"chance\"\": 0.3771328842719901, \"\"structure\"\": \"\"Village\"\", \"\"container\"\": \"\"Mason's chest\"\" }\"\n\n    Bedrock Edition\n\n    Stone Bricks,Village,Mason's\u00a0chest,1,\"37.7% { \"\"item\"\": \"\"Stone Bricks\"\", \"\"stacksize\"\": 1, \"\"chance\"\": 0.3771328842719901, \"\"structure\"\": \"\"Village\"\", \"\"container\"\": \"\"Mason's chest\"\" }\"\n\n\n  Crafting\n    Ingredients,Crafting recipe,Description\n\n    Stone,\"[[None, None, None], ['Stone', 'Stone', None], ['Stone', 'Stone', None]]\",\n\n\n  Stonecutting\n    Ingredients,Cutting recipe,Description\n\n    Stone,,\n\n\n  Smelting\n    Name,Ingredients,Smelting recipe\n\n    Cracked Stone Bricks,Stone Bricks + Any fuel,0.1\n\n\nUsage\n    As stone bricks offer no real advantage over stone or cobblestone, their main use is for decoration.\n\n  Crafting ingredient\n    Name,Ingredients,Crafting recipe\n\n    Mossy Stone Bricks,Stone Bricks + Moss Block,\"[[None, None, None], ['', 'Moss Block', None], [None, None, None]]\"\n\n    Mossy Stone Bricks,Stone Bricks + Vines,\"[[None, None, None], ['', 'Vines', None], [None, None, None]]\"\n\n    Stone Brick Slab,Stone Bricks,\"[[None, None, None], [None, None, None], ['', '', '']]\"\n\n    Stone Brick Stairs,Stone Bricks,\"[['', None, None], ['', '', None], ['', '', '']]\"\n\n    Stone Brick Wall,Stone Bricks,\"[[None, None, None], ['', '', ''], ['', '', '']]\"\n\n\n  Stonecutting\n    Name,Ingredients,Cutting recipe,Description\n\n    Stone Brick Slab or Stone Brick Stairs or Stone Brick Wall,Stone Bricks,2,\n\n    Chiseled Stone Bricks,Stone Bricks,,\u200c [ Java Edition only ] [1]\n\n    Mossy Stone Brick Slab or Mossy Stone Brick Stairs or Mossy Stone Brick Wall,Mossy Stone Bricks,2,\n\n\n  Note Blocks\n    Stone bricks can be placed under note blocks to produce \"bass drum\" sounds.\n\n  Silverfish\n    Silverfish have the ability to enter and hide in stone bricks, creating an infested block of the corresponding type."
question = "What sound do stone bricks produce when placed under note blocks?"
predicted_answer = ask_question(context, question)
print("Q:", question)
print("A:", predicted_answer)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Q: What sound do stone bricks produce when placed under note blocks?
A: drop nothing . block , stone bricks hardness , 1 . 5 tool , wooden pickaxe breaking time ( sec ) [ a ] default , 7 . 5 wooden , 1 . 15 stone , 0 . 6 iron , 0 . 4 diamond , 0 . 3 netherite , 0 . 25 golden , 0 . 2 infested blocks stone bricks


In [ ]:
pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/

In [ ]:
from bert_score import score
cadidates = ["+ any fuel , 0 . 15 , if a bucket is in the fuel slot at the time the sponge completes its drying , the water drains into the bucket , leaving a bucket of water in the fuel slot . otherwise , the"]
references = ["Sponges can only be found in ocean monuments."]
P, R, F1 = score(cadidates, references, lang="en", verbose=True)
print(P)
print(R)
print(F1)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.06 seconds, 16.02 sentences/sec
tensor([0.8074])
tensor([0.8273])
tensor([0.8172])


In [ ]:
from bert_score import score
questions = df['question'].head(50).to_list()
contexts = df['context'].head(50).to_list()
candidates = []
for i in range (50):
    question = questions[i]
    context = contexts[i]
    predicted_answer = ask_question(context, question)
    print("Q:", question)
    print("A:", predicted_answer)
    candidates.append(predicted_answer)

references = df['answer_text'].head(50).to_list()
P, R, F1 = score(candidates, references, lang="en", verbose=True)
print(P)
print(R)
print(F1)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Q: How can a wet sponge be turned back into a normal sponge?
A: + any fuel , 0 . 15 , " if a bucket is in the fuel slot at the time the sponge completes its drying , the water drains into the bucket , leaving a bucket of water in the fuel slot . otherwise , the
Q: Where can sponges be found in Minecraft?
A: + any fuel , 0 . 15 , " if a bucket is in the fuel slot at the time the sponge completes its drying , the water drains into the bucket , leaving a bucket of water in the fuel slot . otherwise , the
Q: What is the maximum distance a sponge can absorb water in Minecraft?
A: + any fuel , 0 . 15 , " if a bucket is in the fuel slot at the time the sponge completes its drying , the water drains into the bucket , leaving a bucket of water in the fuel slot . otherwise , the
Q: What are the default tools for breaking leaves in Minecraft?
A: , 0 . 05 golden , 0 . 05 shears , 0 . 05 sword , 0 . 2 drop , source , no fortune , fortune i , fortune ii , fortune iii jungle sapling s , jungle leaves

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Q: How are leaves used in redstone mechanisms within Minecraft?
A: , 0 . 05 golden , 0 . 05 shears , 0 . 05 sword , 0 . 2 drop , source , no fortune , fortune i , fortune ii , fortune iii jungle sapling s , jungle leaves , 2 . 5 % (
Q: What is the primary usage of netherrack in the game?
A: terrain , and also generates as part of ruined portals in the overworld . netherrack generates as part of bridge and treasure room bastion remnants . post - generation in bedrock edition , netherrack generates as part of nether portals spawned floating . this is what makes netherrack renewable . crimson or warped nylium transforms into netherrack if a solid , opaque block is directly above it , or when it is covered by snow . [ 1 ] usage netherrack is mostly used as a cheap building block . because of its abundance and softness , it is often used to construct temporary structures that can be used for scaffolding , blockage , or other purposes . when netherrack ' s top is lit on fire , the fire burns

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.07 seconds, 46.53 sentences/sec
tensor([0.8273, 0.8032, 0.8104, 0.7353, 0.7876, 0.7512, 0.7771, 0.7690, 0.8176,
        0.8207, 0.7570, 0.7845, 0.7214, 0.7231, 0.6787, 0.7506, 0.7642, 0.7157,
        0.8008, 0.8011, 0.7803, 0.7475, 0.7344, 0.7569, 0.8028, 0.0000, 0.8066,
        0.8037, 0.0000, 0.0000, 0.8297, 0.6975, 0.7527, 0.7487, 0.8078, 0.7629,
        0.7852, 0.8074, 0.8389, 0.8143, 0.7289, 0.0000, 0.7325, 0.7263, 0.7252,
        0.0000, 0.0000, 0.0000, 0.7662, 0.0000])
tensor([0.8826, 0.8258, 0.8300, 0.8274, 0.8358, 0.8042, 0.8963, 0.8453, 0.8050,
        0.8786, 0.8602, 0.8346, 0.7632, 0.8106, 0.8226, 0.8465, 0.8283, 0.8261,
        0.8207, 0.8438, 0.8505, 0.8569, 0.8298, 0.8470, 0.8305, 0.0000, 0.8426,
        0.8277, 0.0000, 0.0000, 0.8432, 0.8351, 0.9117, 0.8365, 0.9441, 0.8577,
        0.8263, 0.8623, 0.8333, 0.8139, 0.9026, 0.0000, 0.8204, 0.8444, 0.8135,
        0.0000, 0.0000, 0.0000, 0.8186, 0.0000])
tensor([0.8540, 0.8144, 0.8201, 0.7786, 0.8110, 0.7768, 0.83

In [ ]:
questions[6:10]

['What is the primary usage of netherrack in the game?',
 'How can netherrack be obtained apart from mining in the Nether?',
 'What happens to netherrack when placed under a note block?',
 'What happens to snow when it is destroyed with a shovel not enchanted with Silk Touch in Java Edition?']

In [ ]:
candidates[6:10]

['silk touch . natural generation netherrack makes up most of the nether \' s terrain , and also generates as part of ruined portals in the overworld . netherrack generates as part of bridge and treasure room bastion remnants . post - generation in bedrock edition , netherrack generates as part of nether portals spawned floating . this is what makes netherrack renewable . crimson or warped nylium transforms into netherrack if a solid , opaque block is directly above it , or when it is covered by snow . [ 1 ] usage netherrack is mostly used as a cheap building block . because of its abundance and softness , it is often used to construct temporary structures that can be used for scaffolding , blockage , or other purposes . when netherrack \' s top is lit on fire , the fire burns indefinitely . unlike soul fire , this fire can spread . using bone meal on netherrack placed next to warped nylium or crimson nylium transforms the netherrack into more warped or crimson nylium . analyzing the n

In [ ]:
references[:5]

['A wet sponge can be dried in a furnace, making the sponge reusable.',
 'Sponges can only be found in ocean monuments.',
 'A sponge absorbs both flowing and source blocks of water up to 7 blocks away',
 'Hoes are the default tools for breaking leaves.',
 'Leaves are affected by the biome dyeing algorithm.']

In [ ]:
print(P.mean())
print(R.mean())
print(F1.mean())

tensor(0.6471)
tensor(0.7067)
tensor(0.6750)
